# Computational Theory Problems - SHA-256 Implementation

## Introduction

This notebook implements the SHA-256 cryptographic hash algorithm from scratch, following the Secure Hash Standard (FIPS PUB 180-4). The implementation is built incrementally across five problems, each building upon the previous work to create a complete working hash function.

### Overview of Problems

1. **Binary Operations**: Implements the seven core bitwise functions used in SHA-256
2. **Constant Generation**: Derives the 64 K constants from cube roots of prime numbers
3. **Message Padding**: Prepares messages for processing in 512-bit blocks
4. **Hash Compression**: Implements the main compression function that processes each block
5. **Password Cracking**: Demonstrates security implications by cracking password hashes

### Purpose

The goal is to understand both the mathematical foundations of cryptographic hashing and its practical applications in security. By implementing SHA-256 step-by-step, we gain insight into how modern cryptographic systems protect data integrity and why proper password hashing techniques are essential.

### References

- **FIPS PUB 180-4**: Secure Hash Standard - Official SHA-256 specification from NIST
- **Section numbers** referenced throughout correspond to the SHS Standard document

---

## Problem 1: Binary Words and Operations

### Overview

This problem implements the fundamental bitwise operations used in SHA-256. These functions are the building blocks of the compression function that processes each 512-bit block of data.

The seven functions implemented are:

**Logical Functions (operate on three 32-bit words):**
- **Parity(x, y, z)**: Returns XOR of all three inputs - used in SHA-1
- **Ch(x, y, z)**: "Choose" function - x chooses bits from y or z based on its bit values
- **Maj(x, y, z)**: "Majority" function - returns 1 if at least 2 of 3 bits are 1

**Sigma Functions (for compression function):**
- **Sigma0(x)**: Uppercase Σ₀ - transforms working variable 'a' using rotations 2, 13, 22
- **Sigma1(x)**: Uppercase Σ₁ - transforms working variable 'e' using rotations 6, 11, 25

**Sigma Functions (for message schedule):**
- **sigma0(x)**: Lowercase σ₀ - uses rotations 7, 18 and shift 3
- **sigma1(x)**: Lowercase σ₁ - uses rotations 17, 19 and shift 10

**Helper Functions:**
- **ROTR(x, n)**: Rotate right - bits wrap around (circular shift)
- **SHR(x, n)**: Shift right - bits are lost (filled with zeros)

All functions use 32-bit unsigned integers (UINT32) to ensure proper overflow behavior.

---

In [28]:
import numpy as np
np.seterr(over='ignore')
# Configure numpy to work with 32-bit unsigned integers
UINT32 = np.uint32

def Parity(x, y, z):
    """
    Parity function: (x ⊕ y ⊕ z)
    
    Reference: SHS Standard (FIPS PUB 180-4), Section 4.1.1 (page 10)
    This section defines how Parity is used in the SHA-1 algorithm.
    The Parity function is specified as x XOR y XOR z.
    Used in: SHA-1 algorithm (rounds 20-39 and 60-79)
    
    Computes the bitwise XOR on each bit of three 32-bit words.
    Examples: returns 1 if an odd number of inputs are 1, else returns 0.
    x == 1 y == 1 z == 0 result == 0
    x == 1 y == 0 z == 1 result == 0
    x == 0 y == 1 z == 1 result == 0 
    x == 1 y == 1 z == 1 result == 1

    Parity = 0b0001 = 0b1
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result of x XOR y XOR z
    
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32(x ^ y ^ z)

In [29]:
# Example test for the Parity function
x, y, z = UINT32(0b1101), UINT32(0b1011), UINT32(0b0111)

print("x:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Parity:", bin(int(Parity(x, y, z)))) # Expected output: 0b1 (result of x XOR y XOR z)


x: 0b1101
y: 0b1011
z: 0b111
Parity: 0b1


In [30]:
def Ch(x, y, z):
    """
    Choose function: (x ∧ y) ⊕ (¬x ∧ z)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    This section defines how the Choose function is used in the SHA-256 algorithm.
    The Choose function selects bits from y and z based on the bits of x.
    Used in: SHA-256 compression function
    
    For each bit position:
    - If x bit is 1, choose the corresponding bit from y
    - If x bit is 0, choose the corresponding bit from z
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result where x "chooses" between y and z
    
    Example:
        x = 1100 (chooses y for first 2 bits, z for last 2 bits)
        y = 1010
        z = 0111
        --------
        Result = 1011 (takes 10 from y, 11 from z)
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32((x & y) ^ (~x & z))

In [31]:
# Example test for the Ch function
x, y, z = UINT32(0b1100), UINT32(0b1010), UINT32(0b0111)

print("\nx:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Ch:", bin(int(Ch(x, y, z)))) # Expected output: 0b1011


x: 0b1100
y: 0b1010
z: 0b111
Ch: 0b1011


In [32]:
def Maj(x, y, z):
    """
    Majority function: (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    This section defines how the Majority function is used in the SHA-256 algorithm.
    The Majority function returns the majority bit for each bit position among x, y, and z
    Used in: SHA-256 compression function
    
    For each bit position:
    - Returns 1 if at least 2 of the 3 bits are 1
    - Returns 0 if at least 2 of the 3 bits are 0
    
    Args:
        x, y, z: 32-bit words
    
    Returns:
        32-bit result representing the majority vote of each bit position
    
    Example:
        x = 1100
        y = 1010
        z = 1001
        --------
        Bit 0: 1,1,1 → majority is 1
        Bit 1: 1,0,0 → majority is 0
        Bit 2: 0,1,0 → majority is 0
        Bit 3: 0,0,1 → majority is 0
        Result = 1000
    """
    x, y, z = UINT32(x), UINT32(y), UINT32(z)
    return UINT32((x & y) ^ (x & z) ^ (y & z))

In [33]:
x, y, z = UINT32(0b1100), UINT32(0b1010), UINT32(0b1001)

print("\nx:", bin(int(x)))
print("y:", bin(int(y)))
print("z:", bin(int(z)))
print("Maj:", bin(int(Maj(x, y, z)))) # Expected output: 0b1000


x: 0b1100
y: 0b1010
z: 0b1001
Maj: 0b1000


In [34]:
def ROTR(x, n, w=32):
    """
    Rotate right (circular right shift) operation.
    
    Reference: SHS Standard, Section 2.2.2 (page 5)
    This section defines the ROTR operation used in SHA algorithms.
    ROTR is used in various SHA functions to achieve bit diffusion.
    
    Args:
        x: w-bit word (uint32)
        n: number of positions to rotate (0 <= n < w)
        w: word size in bits (default: 32)
    
    Returns:
        Result of rotating x right by n positions
    """
    x = UINT32(x)
    n = int(n) % w
    return UINT32((x >> n) | (x << (w - n)))

In [35]:
def Sigma0(x):
    """
    Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Upper case Sigma 0 for SHA-256
    Used in: SHA-256 compression function (transforms working variable 'a')
    
    Takes the input x and:
    1. Rotates it right by 2 positions
    2. Rotates it right by 13 positions
    3. Rotates it right by 22 positions
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the Σ₀ transformation
    
    Example:
        If x = 0xABCD1234
        - ROTR²(x) = rotate right 2
        - ROTR¹³(x) = rotate right 13
        - ROTR²²(x) = rotate right 22
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

In [36]:
x = UINT32(0xABCD1234)

print("\nx:", hex(int(x)))
print("ROTR²(x):", hex(int(ROTR(x, 2))))
print("ROTR¹³(x):", hex(int(ROTR(x, 13))))
print("ROTR²²(x):", hex(int(ROTR(x, 22))))
print("Sigma0:", hex(int(Sigma0(x)))) # Expected output: 0x8f1ec84a


x: 0xabcd1234
ROTR²(x): 0x2af3448d
ROTR¹³(x): 0x91a55e68
ROTR²²(x): 0x3448d2af
Sigma0: 0x8f1ec84a


In [37]:
def Sigma1(x):
    """
    Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Upper case Sigma 1 for SHA-256
    Used in: SHA-256 compression function (transforms working variable 'e')
    
    
    
    
    Takes the input x and:
    1. Rotates it right by 6 positions
    2. Rotates it right by 11 positions
    3. Rotates it right by 25 positions
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the Σ₁ transformation
    
    Example:
        If x = 0x9A8B7C6D
        - ROTR⁶(x) = rotate right 6
        - ROTR¹¹(x) = rotate right 11
        - ROTR²⁵(x) = rotate right 25
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

In [38]:
# Example test for the Sigma1 function
x = UINT32(0x9A8B7C6D)

print("\nx:", hex(int(x)))
print("ROTR⁶(x):", hex(int(ROTR(x, 6))))
print("ROTR¹¹(x):", hex(int(ROTR(x, 11))))
print("ROTR²⁵(x):", hex(int(ROTR(x, 25))))
print("Sigma1:", hex(int(Sigma1(x)))) # Expected output: 0x7e674a53


x: 0x9a8b7c6d
ROTR⁶(x): 0xb66a2df1
ROTR¹¹(x): 0x8db3516f
ROTR²⁵(x): 0x45be36cd
Sigma1: 0x7e674a53


In [39]:
def SHR(x, n):
    """
    Right shift operation.
    
    Reference: SHS Standard, Section 2.2.2 (page 6)
    Definition: SHR_n(x) = x >> n
    
    Args:
        x: w-bit word (uint32)
        n: number of positions to shift (0 <= n < w)
    
    Returns:
        Result of shifting x right by n positions
    """
    return UINT32(UINT32(x) >> n)

In [40]:
def sigma0(x):
    """
    σ₀²⁵⁶(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Lower case sigma 0 for SHA-256

    
    Takes the input x and:
    1. Rotates it right by 7 positions
    2. Rotates it right by 18 positions
    3. SHIFTS it right by 3 positions
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the σ₀ transformation
    
    
    Example:
        If x = 0x12345678
        - ROTR⁷(x) = rotate right 7 (bits wrap)
        - ROTR¹⁸(x) = rotate right 18 (bits wrap)
        - SHR³(x) = shift right 3 (bits lost!)
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))


In [41]:
# Example test for the sigma0 function
x = UINT32(0x12345678)

print("\nx:", hex(int(x)))
print("ROTR⁷(x):", hex(int(ROTR(x, 7))))
print("ROTR¹⁸(x):", hex(int(ROTR(x, 18))))
print("SHR³(x):", hex(int(SHR(x, 3))))
print("sigma0:", hex(int(sigma0(x))))


x: 0x12345678
ROTR⁷(x): 0xf02468ac
ROTR¹⁸(x): 0x159e048d
SHR³(x): 0x2468acf
sigma0: 0xe7fce6ee


In [42]:
def sigma1(x):
    """
    σ₁²⁵⁶(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    Reference: SHS Standard, Section 4.1.2 (page 10)
    Definition: Lower case sigma 1 for SHA-256
    
    Takes the input x and:
    1. Rotates it right by 17 positions
    2. Rotates it right by 19 positions
    3. SHIFTS it right by 10 positions 
    4. XORs all three results together
    
    Args:
        x: 32-bit word
    
    Returns:
        32-bit result of the σ₁ transformation
    
    
    Example:
        If x = 0xFEDCBA98
        - ROTR¹⁷(x) = rotate right 17 (bits wrap)
        - ROTR¹⁹(x) = rotate right 19 (bits wrap)
        - SHR¹⁰(x) = shift right 10 (bits lost!)
        - Result = all three XORed together
    """
    x = UINT32(x)
    return UINT32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

In [43]:
x = UINT32(0xFEDCBA98)

print("\nx:", hex(int(x)))
print("ROTR¹⁷(x):", hex(int(ROTR(x, 17))))
print("ROTR¹⁹(x):", hex(int(ROTR(x, 19))))
print("SHR¹⁰(x):", hex(int(SHR(x, 10))))
print("sigma1:", hex(int(sigma1(x)))) 


x: 0xfedcba98
ROTR¹⁷(x): 0x5d4c7f6e
ROTR¹⁹(x): 0x97531fdb
SHR¹⁰(x): 0x3fb72e
sigma1: 0xca20d79b


## Problem 2: Fractional Parts of Cube Roots

### Overview

This problem generates the 64 round constants (K₀ through K₆₃) used in SHA-256's compression function. These constants are derived from the cube roots of the first 64 prime numbers, providing "nothing up my sleeve" numbers that are verifiably not chosen to hide backdoors.

### Process

1. **Generate the first 64 prime numbers** using trial division
   - Primes: 2, 3, 5, 7, 11, 13, ..., 307, 311

2. **Calculate cube roots** of each prime number
   - Example: ∛2 = 1.25992104989...

3. **Extract fractional parts** (remove integer portion)
   - Example: 1.25992104989... → 0.25992104989...

4. **Take first 32 bits** of the fractional part
   - Multiply by 2³² and convert to integer
   - Example: 0.25992... × 4,294,967,296 = 1,116,352,408
   - Convert to hex: 0x428a2f98

5. **Verify against standard** - All 64 constants must match FIPS PUB 180-4

These constants ensure the hash function has no mathematical weaknesses built in, as they're derived from well-known mathematical properties rather than arbitrary choices.

---

In [44]:
def primes(n):
    """
    Generate the first n prime numbers.
    
    Reference: SHS Standard, Section 4.2.2 (page 11)
    These primes are used to generate the K constants for SHA-256.
    
    Args:
        n: Number of prime numbers to generate
    
    Returns:
        List of the first n prime numbers
    
    """
    if n <= 0:
        return []
    
    prime_list = []
    candidate = 2
    
    while len(prime_list) < n:
        is_prime = True
        
        # Check if candidate is divisible by any prime we've found so far
        for prime in prime_list:
            if prime * prime > candidate:
                break
            if candidate % prime == 0:
                is_prime = False
                break
        
        if is_prime:
            prime_list.append(candidate)
        
        candidate += 1
    
    return prime_list

In [45]:
# Test 1: First 10 primes
print("\nTest 1: First 10 primes")
first_10 = primes(10)
print(f"primes(10) = {first_10}")
print(f"Expected:    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]")


Test 1: First 10 primes
primes(10) = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]
Expected:    [2, 3, 5, 7, 11, 13, 17, 19, 23, 29]


In [46]:
def calculate_cube_roots(num_primes):
    """
    Calculate cube roots of the first num_primes prime numbers.
    
    Reference: SHS Standard, Section 4.2.2 (page 11)
    
    Args:
        num_primes: int
            How many primes to calculate cube roots for
    
    Returns:
        np.ndarray
            Array containing cube roots of first num_primes primes
    """
    # Get the prime numbers
    prime_numbers = primes(num_primes)
    
    # Calculate cube root for each prime
    roots = np.cbrt(prime_numbers)
    
    return roots

In [47]:
# Test 2: First 64 primes and their cube roots
first_64_primes = primes(64)
print("First 64 primes:")
print(first_64_primes)

print("\nCube roots of first 64 primes:")
cube_root_values = calculate_cube_roots(64)
print(cube_root_values)

First 64 primes:
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311]

Cube roots of first 64 primes:
[1.25992105 1.44224957 1.70997595 1.91293118 2.22398009 2.35133469
 2.57128159 2.66840165 2.84386698 3.07231683 3.14138065 3.33222185
 3.44821724 3.50339806 3.60882608 3.75628575 3.89299642 3.93649718
 4.0615481  4.14081775 4.1793392  4.29084043 4.36207067 4.4647451
 4.59470089 4.65700951 4.68754815 4.7474594  4.77685618 4.83458813
 5.0265257  5.07875308 5.15513674 5.18010147 5.30145919 5.32507402
 5.39469071 5.46255557 5.50687845 5.57205466 5.63574079 5.65665283
 5.75896522 5.77899657 5.81864787 5.83827246 5.95334181 6.06412699
 6.1001702  6.11803317 6.15344949 6.20582179 6.22308425 6.30799355
 6.35786118 6.40695858 6.45531481 6.47127363 6.51868392 6

In [48]:
def convert_to_hex_constants(cube_root_array):
    """
    Extract first 32 bits of fractional parts and convert to hexadecimal.
    
    Reference: SHS Standard, Section 4.2.2 (page 11)
    """
    hex_constants = []
    
    for root_value in cube_root_array:
        # Isolate fractional portion
        frac_only = root_value - np.floor(root_value)
        
        # Shift by 32 bits
        bit_shifted = frac_only * (2 ** 32)
        
        # Convert to integer
        as_integer = int(bit_shifted)
        
        # Convert to hex
        hex_constants.append(hex(as_integer))
    
    return hex_constants

In [49]:
#Generate and display
cube_root_array = calculate_cube_roots(64)
hex_k_values = convert_to_hex_constants(cube_root_array)

print(hex_k_values)

['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5', '0xd807aa98', '0x12835b01', '0x243185be', '0x550c7dc3', '0x72be5d74', '0x80deb1fe', '0x9bdc06a7', '0xc19bf174', '0xe49b69c1', '0xefbe4786', '0xfc19dc6', '0x240ca1cc', '0x2de92c6f', '0x4a7484aa', '0x5cb0a9dc', '0x76f988da', '0x983e5152', '0xa831c66d', '0xb00327c8', '0xbf597fc7', '0xc6e00bf3', '0xd5a79147', '0x6ca6351', '0x14292967', '0x27b70a85', '0x2e1b2138', '0x4d2c6dfc', '0x53380d13', '0x650a7354', '0x766a0abb', '0x81c2c92e', '0x92722c85', '0xa2bfe8a1', '0xa81a664b', '0xc24b8b70', '0xc76c51a3', '0xd192e819', '0xd6990624', '0xf40e3585', '0x106aa070', '0x19a4c116', '0x1e376c08', '0x2748774c', '0x34b0bcb5', '0x391c0cb3', '0x4ed8aa4a', '0x5b9cca4f', '0x682e6ff3', '0x748f82ee', '0x78a5636f', '0x84c87814', '0x8cc70208', '0x90befffa', '0xa4506ceb', '0xbef9a3f7', '0xc67178f2']


In [50]:
def check_against_standard(my_generated_values):
    """
    Verify generated K constants against official SHA-256 values.
    
    Reference: SHS Standard, Section 4.2.2 (page 11)
    
    Returns:
        bool - True if all match, False otherwise
    """
    # Official SHA-256 K constants from the standard
    official_k_values = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
        0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
        0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
        0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
        0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
        0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
        0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
        0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
        0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]
    
    # Check each constant
    is_valid = True
    for i in range(64):
        if my_generated_values[i] == hex(official_k_values[i]):
            print(f"K[{i}] MATCH")
        else:
            print(f"K[{i}] MISMATCH")
            is_valid = False
    
    return print("All values match" if is_valid else "Some values don't match")


# Run the verification
cube_root_array = calculate_cube_roots(64)
hex_k_values = convert_to_hex_constants(cube_root_array)

check_against_standard(hex_k_values)

K[0] MATCH
K[1] MATCH
K[2] MATCH
K[3] MATCH
K[4] MATCH
K[5] MATCH
K[6] MATCH
K[7] MATCH
K[8] MATCH
K[9] MATCH
K[10] MATCH
K[11] MATCH
K[12] MATCH
K[13] MATCH
K[14] MATCH
K[15] MATCH
K[16] MATCH
K[17] MATCH
K[18] MATCH
K[19] MATCH
K[20] MATCH
K[21] MATCH
K[22] MATCH
K[23] MATCH
K[24] MATCH
K[25] MATCH
K[26] MATCH
K[27] MATCH
K[28] MATCH
K[29] MATCH
K[30] MATCH
K[31] MATCH
K[32] MATCH
K[33] MATCH
K[34] MATCH
K[35] MATCH
K[36] MATCH
K[37] MATCH
K[38] MATCH
K[39] MATCH
K[40] MATCH
K[41] MATCH
K[42] MATCH
K[43] MATCH
K[44] MATCH
K[45] MATCH
K[46] MATCH
K[47] MATCH
K[48] MATCH
K[49] MATCH
K[50] MATCH
K[51] MATCH
K[52] MATCH
K[53] MATCH
K[54] MATCH
K[55] MATCH
K[56] MATCH
K[57] MATCH
K[58] MATCH
K[59] MATCH
K[60] MATCH
K[61] MATCH
K[62] MATCH
K[63] MATCH
All values match


## Problem 3: Padding

### Overview

This problem implements message padding and block parsing according to the SHA-256 specification. Messages must be padded to ensure their length is a multiple of 512 bits (64 bytes) before hashing.

### Padding Rules (Section 5.1.1 of SHS Standard)

The padding process consists of three steps:

1. **Append a '1' bit** (0x80 in hexadecimal = 10000000 in binary)
   - Marks the end of the actual message

2. **Append '0' bits** until message length ≡ 448 (mod 512)
   - Leaves exactly 64 bits (8 bytes) at the end of the final block
   - May require wrapping to a new 512-bit block

3. **Append original message length** as a 64-bit big-endian integer
   - Stores the length in bits (not bytes)
   - Takes up the final 64 bits of the last block

### Example: Padding "abc"

- Original message: `abc` (3 bytes = 24 bits)
- After adding 0x80: `abc` + `10000000` (4 bytes)
- Add zeros: 52 zero bytes (to reach 56 bytes total)
- Add length: 8 bytes representing 24 in binary
- **Result**: Exactly 64 bytes (512 bits) - one complete block

### Generator Function

The `block_parse()` function is implemented as a generator that yields 512-bit blocks one at a time, which is memory-efficient for processing large messages.

---

In [51]:
def block_parse(msg):
    """
    Generator function that yields 512-bit blocks of a padded message.
    
    Reference: SHS Standard, Sections 5.1.1 and 5.2.1 (pages 13-14)
    
    Padding process:
    1. Append bit '1' to message
    2. Append '0' bits until length ≡ 448 (mod 512)
    3. Append original message length as 64-bit big-endian integer
    
    Args:
        msg: bytes object to be padded and parsed
    
    Yields:
        bytes: 512-bit (64-byte) blocks
    """
    # Store original message length in bits before modification
    original_bit_length = len(msg) * 8
    
    # Create mutable copy for padding
    message_data = bytearray(msg)
    
    # Append '1' bit (0x80 = 10000000 in binary)
    # Marks end of actual message and start of padding
    message_data.append(0x80)
    
    # Calculate how many zero bytes needed to reach 448 bits (56 bytes) in current block
    # The % 64 handles wrapping to next block if necessary
    bytes_in_block = len(message_data) % 64
    zeros_needed = (56 - bytes_in_block) % 64
    
    # Add zero padding (reserving last 8 bytes for length)
    message_data.extend(b'\x00' * zeros_needed)
    
    # Append original length as 64-bit big-endian integer
    message_data.extend(original_bit_length.to_bytes(8, byteorder='big'))
    
    # Sanity check: total length must be multiple of 512 bits (64 bytes)
    assert len(message_data) % 64 == 0, "Total length must be multiple of 512 bits"
    
    # Generate and yield each 512-bit block
    for block_start in range(0, len(message_data), 64):
        yield bytes(message_data[block_start:block_start + 64])

In [52]:
# Test 1: Short message "abc"
print("\nTest 1: msg = b'abc' (24 bits)")
print("-" * 70)
blocks = list(block_parse(b'abc'))
print(f"Number of blocks: {len(blocks)}")
print(f"Block 1 length: {len(blocks[0])} bytes")
print(f"First 10 bytes: {blocks[0][:10].hex()}")
print(f"Last 8 bytes (length): {blocks[0][-8:].hex()}")

# Test 2: Empty message
print("\nTest 2: msg = b'' (0 bits)")
print("-" * 70)
blocks = list(block_parse(b''))
print(f"Number of blocks: {len(blocks)}")
print(f"Block 1 length: {len(blocks[0])} bytes")

# Test 3: Longer message
print("\nTest 3: msg = b'hello world' (88 bits)")
print("-" * 70)
blocks = list(block_parse(b'hello world'))
print(f"Number of blocks: {len(blocks)}")
print(f"Block 1 length: {len(blocks[0])} bytes")

# Test 4: Message that requires 2 blocks
print("\nTest 4: Long message (requires 2 blocks)")
print("-" * 70)
long_msg = b'a' * 60  # 60 bytes = 480 bits
blocks = list(block_parse(long_msg))
print(f"Original message: {len(long_msg)} bytes ({len(long_msg) * 8} bits)")
print(f"Number of blocks: {len(blocks)}")
print(f"Block 1 length: {len(blocks[0])} bytes")
print(f"Block 2 length: {len(blocks[1])} bytes")


Test 1: msg = b'abc' (24 bits)
----------------------------------------------------------------------
Number of blocks: 1
Block 1 length: 64 bytes
First 10 bytes: 61626380000000000000
Last 8 bytes (length): 0000000000000018

Test 2: msg = b'' (0 bits)
----------------------------------------------------------------------
Number of blocks: 1
Block 1 length: 64 bytes

Test 3: msg = b'hello world' (88 bits)
----------------------------------------------------------------------
Number of blocks: 1
Block 1 length: 64 bytes

Test 4: Long message (requires 2 blocks)
----------------------------------------------------------------------
Original message: 60 bytes (480 bits)
Number of blocks: 2
Block 1 length: 64 bytes
Block 2 length: 64 bytes


## Problem 4: Hashes

### Overview

This problem implements the core SHA-256 compression function that processes each 512-bit message block. This is where all the previously implemented functions come together to perform the actual hashing.

### The Compression Function (Section 6.2.2 of SHS Standard)

The hash function processes each block through four main steps:

#### Step 1: Prepare Message Schedule (W)

- Parse the 512-bit block into sixteen 32-bit words (W₀ to W₁₅)
- Extend to 64 words (W₁₆ to W₆₃) using the formula:
```
  Wₜ = σ₁(Wₜ₋₂) + Wₜ₋₇ + σ₀(Wₜ₋₁₅) + Wₜ₋₁₆
```
- This expansion creates diffusion - each input bit affects many output bits

#### Step 2: Initialize Working Variables

- Eight 32-bit working variables: a, b, c, d, e, f, g, h
- Initialized with the current hash value from the previous block
- For the first block, use the initial hash values (H₀ through H₇)

#### Step 3: Perform 64 Rounds

Each round updates the working variables using:
```
T₁ = h + Σ₁(e) + Ch(e,f,g) + Kₜ + Wₜ
T₂ = Σ₀(a) + Maj(a,b,c)

h = g
g = f
f = e
e = d + T₁
d = c
c = b
b = a
a = T₁ + T₂
```

#### Step 4: Compute Intermediate Hash

Add the working variables back to the current hash value:
```
H₀ = H₀ + a
H₁ = H₁ + b
... (and so on for all 8 values)
```

### Result

After processing all blocks, the final hash value (H₀ through H₇) is concatenated to produce the 256-bit SHA-256 digest.

---

In [53]:
def hash(current, block):
    """
    Compute next hash value from current hash and message block.
    
    Reference: SHS Standard, Section 6.2.2 (page 22)
    
    Args:
        current: tuple of 8 32-bit integers
        block: 64 bytes (512 bits)
    
    Returns:
        tuple of 8 updated 32-bit integers
    """
    
    # Step 1: Build message schedule (64 words)
    W = []
    
    # First 16 words from block
    for i in range(16):
        W.append(int.from_bytes(block[i*4:i*4+4], byteorder='big'))
    
    # Extend to 64 words
    for i in range(16, 64):
        W.append((sigma1(W[i-2]) + W[i-7] + sigma0(W[i-15]) + W[i-16]) & 0xFFFFFFFF)
    
    # Step 2: Initialize working variables
    a, b, c, d, e, f, g, h = current
    
    # Step 3: 64 rounds
    for i in range(64):
        T1 = (h + Sigma1(e) + Ch(e, f, g) + K[i] + W[i]) & 0xFFFFFFFF
        T2 = (Sigma0(a) + Maj(a, b, c)) & 0xFFFFFFFF
        
        h = g
        g = f
        f = e
        e = (d + T1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (T1 + T2) & 0xFFFFFFFF
    
    # Step 4: Add to current hash
    return (
        (current[0] + a) & 0xFFFFFFFF,
        (current[1] + b) & 0xFFFFFFFF,
        (current[2] + c) & 0xFFFFFFFF,
        (current[3] + d) & 0xFFFFFFFF,
        (current[4] + e) & 0xFFFFFFFF,
        (current[5] + f) & 0xFFFFFFFF,
        (current[6] + g) & 0xFFFFFFFF,
        (current[7] + h) & 0xFFFFFFFF
    )
       
        

## Problem 5: Passwords

### Overview

This problem demonstrates the complete SHA-256 implementation by using it to crack password hashes. It also highlights critical security vulnerabilities when SHA-256 is used improperly for password storage.

### The Challenge

Given three SHA-256 password hashes, determine the original passwords:
1. `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
2. `873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34`
3. `b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342`

### Attack Method: Dictionary Attack

A dictionary attack works by:
1. Maintaining a list of common passwords
2. Hashing each candidate password with SHA-256
3. Comparing the computed hash against the target hash
4. When hashes match, the password is found

This is effective because:
- Users often choose predictable passwords
- SHA-256 is extremely fast (billions of hashes per second on modern hardware)
- No computational barrier slows down the attacker

---

In [54]:
# Problem 5: Password Cracking

# Convert hex strings to integers for use in hash function
K = [int(h, 16) for h in hex_k_values]

# Initial hash values from SHA-256 standard (page 11)
INITIAL_HASH = (
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
)

def my_sha256(message_string):
    """
    Complete SHA-256 implementation using functions from Problems 1-4.
    
    Args:
        message_string: str - the message to hash
    
    Returns:
        str - hexadecimal hash string
    """
    # Convert string to bytes (UTF-8 encoding)
    msg_bytes = message_string.encode('utf-8')
    
    # Initialize with starting hash values
    current_hash = INITIAL_HASH
    
    # Process each 512-bit block
    for block in block_parse(msg_bytes):
        current_hash = hash(current_hash, block)
    
    # Convert final hash to hexadecimal string
    result = ''.join(f'{h:08x}' for h in current_hash)
    return result


def crack_passwords():
    """
    Crack three SHA-256 password hashes using dictionary attack.
    
    Returns
    -------
    dict
        Mapping of password number to cracked password
    """
    # Target hashes to crack
    target_hashes = {
        1: '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8',
        2: '873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34',
        3: 'b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342'
    }
    
    # Common password dictionary
    password_list = [
    'password', '123456', '12345678', 'qwerty', 'abc123',
    'monkey', '1234567', 'letmein', 'trustno1', 'dragon',
    'baseball', 'iloveyou', 'master', 'sunshine', 'ashley',
    'bailey', 'shadow', '123123', '654321', 'superman',
    'princess', 'michael', 'football', 'welcome', 'jesus',
    'ninja', 'mustang', 'P@ssw0rd', 'password1', 'cheese', 'computer',
    'admin', 'root', 'test', 'guest', 'hello',
    'login', 'passw0rd', 'pass', '1234', '12345',
    'summer', 'winter', 'spring', 'fall', 'love',
    'secret', 'blue', 'green', 'red', 'black',
    '111111', '121212', '000000', '666666', '123321',
    'qwertyuiop', 'asdfghjkl', '1q2w3e4r', '1qaz2wsx', 'qazwsx',
    '1234567890', '123456789', 'Password', 'password123',
    'abc', 'abcd', 'abcde', 'abcdef', 'abcdefg',
    '123', '1234', '123457', '1234568', '12345679',
    'letmein', 'welcome', 'solo', 'access', 'charlie',
    'flower', 'pepper', 'cookie', 'starwars', 'whatever',
    '1q2w3e', 'qwe123', 'zxcvbnm', 'asdfghjkl', 'master',
    'hello123', 'welcome123', 'login', 'admin123', 'root123'
]
    
    cracked = {}
    
    for num, target in target_hashes.items():
        print(f"\nCracking password {num}...")
        found = False
        
        for pwd in password_list:
            # Hash the candidate password 
            hashed = my_sha256(pwd)
            
            if hashed == target:
                cracked[num] = pwd
                print(f"✓ Password {num} found: '{pwd}'")
                found = True
                break
        
        if not found:
            print(f"✗ Password {num} not found in dictionary")
    
    print(f"\n{'='*50}")
    print(f"Cracked {len(cracked)}/3 passwords")
    print(f"{'='*50}")
    
    return cracked


# Test SHA-256 works correctly first
print("Testing SHA-256 implementation:")
test_hash = my_sha256("password")
print(f"SHA256('password') = {test_hash}")
print(f"Expected:            5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8")
print(f"Match: {test_hash == '5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8'}")

# Crack the passwords
results = crack_passwords()
print("\nResults:", results)

Testing SHA-256 implementation:
SHA256('password') = 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Expected:            5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Match: True

Cracking password 1...
✓ Password 1 found: 'password'

Cracking password 2...
✓ Password 2 found: 'cheese'

Cracking password 3...
✓ Password 3 found: 'P@ssw0rd'

Cracked 3/3 passwords

Results: {1: 'password', 2: 'cheese', 3: 'P@ssw0rd'}


### Security Improvements

**NEVER use raw SHA-256 for password storage.** Here are the essential security measures:

#### 1. Use a Cryptographic Salt

A salt is random data added to each password before hashing:
```python
# Instead of: hash(password)
# Do this:    hash(password + random_salt)
```

**Benefits:**
- Each user gets a unique salt (stored alongside the hash)
- Same password produces different hashes for different users
- Rainbow tables become useless
- Forces attackers to crack each password individually

**Example:**
```
User 1: hash("password" + "xK9mP2qR") = abc123...
User 2: hash("password" + "7nBvC4tY") = def456...
```

**Reference:** NIST Special Publication 800-63B - Digital Identity Guidelines [1]

#### 2. Use Purpose-Built Password Hashing Functions

Replace SHA-256 with algorithms designed for password storage:

**bcrypt** (Industry Standard)
- Intentionally slow (configurable work factor)
- Built-in salt generation
- Time-tested and widely adopted
- Based on the Blowfish cipher
```python
import bcrypt
hashed = bcrypt.hashpw(password.encode(), bcrypt.gensalt(rounds=12))
```

**Reference:** Provos, N., & Mazières, D. (1999). "A Future-Adaptable Password Scheme" [2]

**Argon2** (Modern Recommended)
- Winner of Password Hashing Competition (2015)
- Memory-hard (resists GPU/ASIC attacks)
- Configurable time, memory, and parallelism costs
- Three variants: Argon2d, Argon2i, Argon2id (recommended)
```python
from argon2 import PasswordHasher
ph = PasswordHasher()
hashed = ph.hash(password)
```

**Reference:** Biryukov, A., Dinu, D., & Khovratovich, D. (2016). "Argon2: The Memory-Hard Function for Password Hashing and Other Applications" [3]

**PBKDF2** (Widely Supported)
- Applies hash function thousands of times
- NIST approved
- Available in most languages
- Recommended minimum 100,000 iterations
```python
from hashlib import pbkdf2_hmac
hashed = pbkdf2_hmac('sha256', password.encode(), salt, 100000)
```

**Reference:** IETF RFC 8018 - PKCS #5: Password-Based Cryptography Specification [4]

#### 3. Enforce Strong Password Policies

- **Minimum length**: 12+ characters
- **Complexity**: Mix of uppercase, lowercase, numbers, symbols
- **Block common passwords**: Reject passwords from known breach databases
- **No password reuse**: Prevent using old passwords

**Reference:** NIST SP 800-63B Section 5.1.1 - Memorized Secret Verifiers [1]

#### 4. Additional Defense Layers

**Pepper (Secret Server-Side Salt)**
- Additional secret value stored separately from database
- Even if database is compromised, passwords remain protected
```python
hash(password + salt + SECRET_PEPPER)
```

**Rate Limiting**
- Limit login attempts (e.g., 5 attempts per hour)
- Exponential backoff after failures
- Slows down brute force attacks

**Multi-Factor Authentication (MFA)**
- Even if password is compromised, account stays secure
- Requires second factor (SMS, authenticator app, hardware key)

**Reference:** OWASP Password Storage Cheat Sheet [5]

---

### References

[1] Grassi, P. A., et al. (2017). *NIST Special Publication 800-63B: Digital Identity Guidelines - Authentication and Lifecycle Management*. National Institute of Standards and Technology. https://pages.nist.gov/800-63-3/sp800-63b.html

[2] Provos, N., & Mazières, D. (1999). *A Future-Adaptable Password Scheme*. Proceedings of the 1999 USENIX Annual Technical Conference. https://www.usenix.org/legacy/events/usenix99/provos/provos.pdf

[3] Biryukov, A., Dinu, D., & Khovratovich, D. (2016). *Argon2: The Memory-Hard Function for Password Hashing and Other Applications*. https://github.com/P-H-C/phc-winner-argon2/blob/master/argon2-specs.pdf

[4] Moriarty, K., et al. (2017). *PKCS #5: Password-Based Cryptography Specification Version 2.1*. IETF RFC 8018. https://tools.ietf.org/html/rfc8018

[5] OWASP Foundation. (2023). *Password Storage Cheat Sheet*. https://cheatsheetseries.owasp.org/cheatsheets/Password_Storage_Cheat_Sheet.html

---

### Key Takeaways

1. **SHA-256 is NOT secure for password storage** - it's too fast
2. **Always use salts** - unique random salt per password
3. **Use bcrypt or Argon2** - purpose-built for password hashing
4. **Defense in depth** - combine multiple security measures

Modern password security requires multiple layers of protection. The ease with which these passwords were cracked demonstrates why proper password hashing is critical for protecting user accounts.

---

---

## Conclusion

This notebook successfully implemented the complete SHA-256 cryptographic hash algorithm from scratch, following the FIPS PUB 180-4 Secure Hash Standard specification. The implementation was built incrementally across five problems, demonstrating both the mathematical foundations and practical security implications of cryptographic hashing.

### Implementation Achievements

Complete SHA-256 Algorithm
- All seven bitwise operations (Parity, Ch, Maj, Sigma0, Sigma1, sigma0, sigma1) implemented and tested
- 64 round constants (K₀-K₆₃) correctly derived from cube roots of prime numbers
- Message padding and block parsing conforming to the standard
- Hash compression function successfully processing 512-bit blocks
- Final hash values matching official test vectors

Verification Against Standard
- All 64 K constants verified to match FIPS PUB 180-4 exactly
- Test hash of "password" matches expected value: `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
- Successfully cracked three password hashes demonstrating working implementation

### Key Learnings

**1. Cryptographic Design Principles**
- SHA-256 uses carefully chosen mathematical constants (from prime cube roots) to ensure no hidden backdoors
- Multiple rounds of bit diffusion ensure each input bit affects many output bits
- Combination of rotation, shifting, and logical operations creates strong one-way function

**2. Security Implications**
- Raw SHA-256 is completely inadequate for password storage
- Fast computation (designed for data integrity) becomes a vulnerability for passwords
- Attackers can test billions of passwords per second using modern hardware
- All three passwords cracked instantly using simple dictionary attack

**3. Proper Password Security Requirements**
- **Salts**: Unique random data per password prevents rainbow table attacks
- **Slow hash functions**: bcrypt, Argon2, or PBKDF2 designed to resist brute force
- **Defense in depth**: Multiple security layers (MFA, rate limiting, strong policies)

### Practical Applications

This implementation demonstrates why:
- Cryptographic hash functions are essential for data integrity verification
- The same algorithm can be secure for one purpose (file hashing) but insecure for another (password storage)
- Understanding the internals of cryptographic systems is crucial for using them correctly
- Security requires proper application of algorithms, not just their existence

---

**End of Notebook**